In [92]:
import Pkg
using CUDA
using Statistics
using Random
using Test
import Base: length, size, iterate, eltype, IteratorSize, IteratorEltype, haslength, @propagate_inbounds, repeat, rand, tail
import .Iterators: cycle, Cycle, take
using Plots; default(fmt=:png,ls=:auto)

import Knet
using Knet: deconv4, conv4, unpool, pool, mat, sigm, KnetArray, nll, zeroone, progress, adam!, sgd!, param, param0, dropout, relu, minibatch, Data
import Knet: train!

using MLDatasets: MNIST

Pkg.activate("Project.toml")

 Activating environment at `~/Desktop/MSc/repos/yolo-julia/Project.toml`


In [93]:
# Load data
xtrn, ytrn = MNIST.traindata(Float32); ytrn[ytrn.==0] .= 10;
xtst, ytst = MNIST.testdata(Float32);  ytst[ytst.==0] .= 10;

In [94]:
include("nn.jl")

import .NN

dtrn = minibatch(xtrn, ytrn, 200; xsize = (28,28,1,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(xtst, ytst, 200; xsize = (28,28,1,:), xtype=Knet.atype());
x, y = first(dtst)


dl2 = NN.Conv2d(5, 1, 1)
println(size(dl2.w))
cc = NN.Chain()
println(typeof(cc))
push!(cc.layers, dl2)
cc.layers

(5, 5, 1, 1)
Main.NN

.Chain


1-element Array{Any,1}:
 Main.NN.Conv2d(P(Array{Float32,4}(5,5,1,1)), nothing, 1, 0)

In [97]:
include("utils/parse_config.jl")
include("models.jl")

mdefs = parse_model_cfg("yolov3.cfg")
layers, routes = create_modules(mdefs, 416)


(Any[Main.NN.Chain(Any[Main.NN.Conv2d(P(Array{Float32,4}(3,3,3,32)), nothing, 1, 1), Main.NN.BatchNorm2d(Knet.Ops20.BNMoments(0.03, nothing, nothing, zeros, ones), Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0001), Main.NN.LeakyReLU(Float32[0.1])]), Main.NN.Chain(Any[Main.NN.Conv2d(P(Array{Float32,4}(3,3,32,64)), nothing, 2, 1), Main.NN.BatchNorm2d(Knet.Ops20.BNMoments(0.03, nothing, nothing, zeros, ones), Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0001), Main.NN.LeakyReLU(Float32[0.1])]), Main.NN.Chain(Any[Main.NN.Conv2d(P(Array{Float32,4}(1,1,64,32)), nothing, 1, 0), Main.NN.BatchNorm2d(Knet.Ops20.BNMoments(0.03, nothing, nothing, zeros, ones), Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0001), Main.NN.LeakyReLU(Float32[0.1])]), Main.NN.Chain(Any[Main.NN.Conv2d(P(Array{Float32

In [103]:
println(routes)

Any[2, 6, 9, 13, 16, 19, 22, 25, 28, 31, 34, 38, 41, 44, 47, 50, 53, 56, 59, 63, 66, 69, 72, 82, 80, 86, 62, 94, 92, 98, 37, 106]


In [64]:
include("nn.jl")

import .NN

In [65]:
function dtype(); CUDA.functional() ? KnetArray{Float32} : Array{Float32}; end;

# Define dense layer:
struct Dense
    w; b; f; p; 
    
    function Dense(inputsize::Int, outputsize::Int, f=relu; 
            pdrop=0, atype=dtype())
        
        return new(
            param(outputsize, inputsize; atype=atype),
            param0(outputsize; atype=atype),
            f,
            pdrop
        )
    end
end

(d::Dense)(x) = d.f.(d.w * mat(dropout(x, d.p)) .+ d.b)


# Define convolutional layer:
struct Conv 
    w; b; f; p; 
    
    function Conv(w1::Int, w2::Int, cx::Int, cy::Int, f=relu;
            pdrop=0, atype=dtype())
        
        return new(
            param(w1,w2,cx,cy; atype=atype), 
            param0(1,1,cy,1; atype=atype),
            f,
            pdrop
        )
    end
end

(c::Conv)(x) = c.f.(conv4(c.w, dropout(x, c.p)) .+ c.b)


# Define chain of layers
struct Chain
    layers
    Chain(layers...) = new(layers)
end

(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)  # Forward pass

# Loss and accuracy
function (c::Chain)(x, y; accuracy::Bool=false)
    y_pred = c(x)
    
    if accuracy
        correct = 0.0
        
        for i=1:length(y)
            correct += y[i] == findmax(y_pred[:, i]; dims=1)[2][1] ? 1.0 : 0.0
        end
        
        return correct / length(y)
    else
        return nll(y_pred, y)
    end
end

(c::Chain)(d::Data; accuracy::Bool=false) = mean(c(x,y; accuracy=accuracy) for (x,y) in d)  # Batch loss

In [76]:
function train!(model::NN.Chain, train_data::Data, test_data::Data;
                  period::Int=4, iters::Int=100, lr=0.15, optimizer=sgd!)  # or optimizer=adam!
    
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    
    for i in 0:period:iters
        push!(train_loss, model(train_data))
        push!(test_loss, model(test_data))
    
        push!(train_acc, model(train_data; accuracy=true))
        push!(test_acc, model(test_data; accuracy=true))
        
        optimizer(model, take(cycle(train_data), period); lr=lr)
        
        
            println("Iter: ", i)
    end
    
    return 0:period:iters, train_loss, train_acc, test_loss, test_acc
end

train! (generic function with 5 methods)

In [77]:


Random.seed!(13)

model = NN.Chain(
    NN.Conv2d(1, 3, 5),
    NN.LeakyReLU(),
    NN.Upsample2d(2),
    NN.Dense(6912,10,identity),
)



mdefs = parse_model_cfg("dummy.cfg")
layers, routes = create_modules(mdefs, 416)
model = NN.Chain(
    layers[1],
    layers[2],
    layers[3],
    NN.Dense(12544,10,identity),
)

# Minibatches
dtrn = minibatch(xtrn, ytrn, 200; xsize = (28,28,1,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(xtst, ytst, 200; xsize = (28,28,1,:), xtype=Knet.atype());

iters, trnloss, trnacc, tstloss, tstacc = train!(
    model, dtrn, dtst; 
    period=1, iters=16, lr=0.15, optimizer=sgd!);

# @time train!(
#     model, dtrn, dtst; 
#     period=1, iters=10, lr=0.15, optimizer=sgd!);

println("Train loss: $(round(trnloss[end], digits=2)), Best train accuracy: $(ceil(maximum(trnacc), digits=2))")
println("Test loss: $(round(tstloss[end], digits=2)), Best test accuracy: $(ceil(maximum(tstacc), digits=2))")

plot(iters, trnloss, label="train", xlabel="Iterations", ylabel="Loss")
display(plot!(iters, tstloss, label="test"))

plot(iters, round.(1 .- trnacc, digits=2), label="train", xlabel="Iterations", ylabel="Misclassification error")
display(plot!(iters, round.(1 .- tstacc, digits=2), label="test"))

[1, 4]
[1, 4, 4]
[1, 4, 4, 4]
Iter: 0
Iter: 1
Iter: 2
Iter: 3


LoadError: TaskFailedException:
InterruptException:
Stacktrace:
 [1] ntuple(::NNlib.var"#4#5"{Tuple{Int64,Int64,Int64},Tuple{Int64,Int64,Int64},Tuple{Int64,Int64,Int64},NTuple{6,Int64},Tuple{Int64,Int64,Int64}}, ::Int64) at ./ntuple.jl:17
 [2] output_size at /home/bcs/.julia/packages/NNlib/E3YZL/src/dim_helpers/ConvDims.jl:120 [inlined]
 [3] im2col!(::SubArray{Float32,2,Array{Float32,3},Tuple{Base.Slice{Base.OneTo{Int64}},Base.Slice{Base.OneTo{Int64}},Int64},true}, ::SubArray{Float32,4,Array{Float32,5},Tuple{Base.Slice{Base.OneTo{Int64}},Base.Slice{Base.OneTo{Int64}},Base.Slice{Base.OneTo{Int64}},Base.Slice{Base.OneTo{Int64}},Int64},true}, ::NNlib.DenseConvDims{3,(3, 3, 1),1,4,(1, 1, 1),(1, 1, 1, 1, 0, 0),(1, 1, 1),false}) at /home/bcs/.julia/packages/NNlib/E3YZL/src/impl/conv_im2col.jl:189
 [4] macro expansion at /home/bcs/.julia/packages/NNlib/E3YZL/src/impl/conv_im2col.jl:53 [inlined]
 [5] (::NNlib.var"#406#threadsfor_fun#177"{Array{Float32,3},Float32,Float32,Array{Float32,5},Array{

In [68]:
xx = convert(Knet.atype(), reshape(1:36, (6, 6, 1, 1)))
# y = dl2(xx)

6×6×1×1 Array{Float32,4}:
[:, :, 1, 1] =
 1.0   7.0  13.0  19.0  25.0  31.0
 2.0   8.0  14.0  20.0  26.0  32.0
 3.0   9.0  15.0  21.0  27.0  33.0
 4.0  10.0  16.0  22.0  28.0  34.0
 5.0  11.0  17.0  23.0  29.0  35.0
 6.0  12.0  18.0  24.0  30.0  36.0

In [79]:
unpool(xx; window=4)

24×24×1×1 Array{Float32,4}:
[:, :, 1, 1] =
 1.0  1.0  1.0  1.0   7.0   7.0   7.0  …  25.0  25.0  31.0  31.0  31.0  31.0
 1.0  1.0  1.0  1.0   7.0   7.0   7.0     25.0  25.0  31.0  31.0  31.0  31.0
 1.0  1.0  1.0  1.0   7.0   7.0   7.0     25.0  25.0  31.0  31.0  31.0  31.0
 1.0  1.0  1.0  1.0   7.0   7.0   7.0     25.0  25.0  31.0  31.0  31.0  31.0
 2.0  2.0  2.0  2.0   8.0   8.0   8.0     26.0  26.0  32.0  32.0  32.0  32.0
 2.0  2.0  2.0  2.0   8.0   8.0   8.0  …  26.0  26.0  32.0  32.0  32.0  32.0
 2.0  2.0  2.0  2.0   8.0   8.0   8.0     26.0  26.0  32.0  32.0  32.0  32.0
 2.0  2.0  2.0  2.0   8.0   8.0   8.0     26.0  26.0  32.0  32.0  32.0  32.0
 3.0  3.0  3.0  3.0   9.0   9.0   9.0     27.0  27.0  33.0  33.0  33.0  33.0
 3.0  3.0  3.0  3.0   9.0   9.0   9.0     27.0  27.0  33.0  33.0  33.0  33.0
 3.0  3.0  3.0  3.0   9.0   9.0   9.0  …  27.0  27.0  33.0  33.0  33.0  33.0
 3.0  3.0  3.0  3.0   9.0   9.0   9.0     27.0  27.0  33.0  33.0  33.0  33.0
 4.0  4.0  4.0  4.0  10.0  10.0  